# 메타데이터에 모델 점수를 연결

In [ ]:
import pandas as pd
import pickle
import numpy as np
from pathlib import Path

norm_path = Path('./experiment/matches_scores/normal')
norm_list = list(norm_path.glob('*'))

zero_path = Path('./experiment/matches_scores/zero_h0')
zero_list = list(zero_path.glob('*'))

with open('./Dataset/meta_analysis.ftr', 'rb') as file:
    meta = pd.read_feather(file)

meta['score_norm'] = np.nan
for idx, norm_file in enumerate(norm_list):
    print(f'[Norm] ({idx:5d}/{len(norm_list)}) {idx/len(norm_list)*100:3.1f}% progressing...', end='\r')
    mat_name = norm_file.name[:-4]

    with norm_file.open('rb') as file:
        scores = pickle.load(file)
    scores = [scores[idx].sum().item() for idx in range(10)]

    meta.loc[(meta.mat_no==mat_name), 'score_norm'] = scores

meta['score_zero'] = np.nan
for idx, zero_file in enumerate(zero_list):
    print(f'[Zero] ({idx:5d}/{len(zero_list)}) {idx/len(zero_list)*100:3.1f}% progressing...', end='\r')
    mat_name = zero_file.name[:-4]

    with zero_file.open('rb') as file:
        scores = pickle.load(file)
    scores = [scores[idx].sum().item() for idx in range(10)]

    meta.loc[(meta.mat_no==mat_name), 'score_zero'] = scores

meta.to_feather('./Dataset/meta_analysis_with_score.ftr')
meta

# 각 매치별 플레이어 KDA, Gold, Creep score, 모델 점수를 순위화한 데이터 생성

In [1]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm

with open('./Dataset/meta_analysis_with_score.ftr', 'rb') as file:
    meta = pd.read_feather(file)
meta.rename(columns={'gold_earned':'gold', 'score_norm':'score_model1', 'score_zero':'score_model2'}, inplace=True)
columns = ['match', 'player', 'kda', 'gold', 'creep', 'score_model1', 'score_model2']

block_num = int(len(meta) / 10)
ar = np.zeros([len(meta), len(columns)], dtype=np.int8)
df = pd.DataFrame(ar, columns=columns)

for idx in tqdm(range(block_num)):
    sdx = idx * 10
    edx = sdx + 9
    tmp = meta.loc[sdx:edx].copy()
    df.loc[sdx:edx, 'match'] = tmp['mat_no']
    df.loc[sdx:edx, 'player'] = tmp['player']

    for col in columns[2:]:
        tmp.sort_values(by=col, axis=0, inplace=True, ignore_index=True, ascending=False)
        data = [int(tmp.index[tmp.player==pi].item()) for pi in range(10)]
        df.loc[sdx:edx, col] = data

tqdm.pandas()
df['avg'] = df.progress_apply((lambda x: int((x.gold+x.creep+x.kda)/3)), axis=1)

df.to_feather('./experiment/analysis_result2.ftr')
df

100%|██████████| 455750/455750 [00:04<00:00, 94189.91it/s]


,match,player,kda,gold,creep,score_model1,score_model2,avg
0,210513_na1_3902040674,0,9,9,5,6,7,7
1,210513_na1_3902040674,1,5,6,6,7,3,5
2,210513_na1_3902040674,2,8,8,4,9,9,6
3,210513_na1_3902040674,3,6,4,3,8,8,4
4,210513_na1_3902040674,4,7,7,7,5,5,7
...,...,...,...,...,...,...,...,...
455745,210517_jp1_302513885,5,1,1,4,1,3,2
455746,210517_jp1_302513885,6,0,2,9,2,0,3
455747,210517_jp1_302513885,7,2,0,0,4,1,0
455748,210517_jp1_302513885,8,5,3,2,3,2,3
